In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import optuna
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import log_loss

In [2]:
# class CFG:
#     train_data_path = "../input/tabular-playground-series-may-2021/train.csv"
#     test_data_path =" ../input/tabular-playground-series-may-2021/test.csv"
#     k_folds = 5
#     #feel free to change this model to anything you want to tune 
#     model = LGBMClassifier()
#     #same as the model feel free to change the params to anything as long as they are for the model 
#     params =  param = {
#         'objective': 'multiclass',
#         'boosting': 'gbdt',
#         'metric': 'multi_logloss',
#         'verbose': -1,
#         'learning_rate': trial.suggest_uniform('learning_rate', 0.000001, 0.01),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 2000, 50),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 512),
#         'max_depth': trial.suggest_int('max_depth', 2, 20),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'max_bin': trial.suggest_int('max_bin', 10, 300, 10),
#     }
#     objective_func = log_loss()
#     #number of trials for optuna to run
#     opt_trials = 100
#     #direction that optuna will take for study 
#     opt_direction = "maximize"

In [3]:
#import in the data 
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [4]:
feature_cols = train_df.columns[train_df.columns.str.startswith('feature_')]
le = LabelEncoder()
le.fit(train_df['target'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
train_df.target = le.transform(train_df.target)
# Since train and test data have different unique values, 
# we concatenate them to one giant dataset for the encoding process
all_data = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

# One Hot Encode target variable
ohex = ce.OneHotEncoder(handle_unknown='value', use_cat_names=True)
OH_features = pd.DataFrame(ohex.fit_transform(all_data[feature_cols].astype(str)))
OH_features

display(OH_features.head())

# Merge new OH encoded target to train_df 
all_data = pd.concat([all_data, OH_features], axis=1)

# Split train and test set again 
train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):].reset_index(drop=True)

{'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}


/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,feature_0_0,feature_0_1,feature_0_9,feature_0_2,feature_0_3,feature_0_4,feature_0_7,feature_0_5,feature_0_6,feature_0_8,...,feature_49_12,feature_49_7,feature_49_13,feature_49_15,feature_49_14,feature_49_16,feature_49_18,feature_49_20,feature_49_19,feature_49_21
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# sk_fold = StratifiedKFold(5)
# splits = sk_fold.split(X,y)

In [6]:
feature_cols = train_df.columns[train_df.columns.str.startswith('feature_')]
X = train_df[feature_cols]
y = train_df['target']
x_train,x_valid,y_train,y_valid = train_test_split(X,y,random_state = 101)
def objective(trial):    
        params = {'iterations':10000,
              "od_wait":1000, 
              "od_type":'Iter',
              "max_ctr_complexity":15,
              'depth': trial.suggest_int("depth", 3, 10),
              'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 0.0001, 25, log=True),
              'grow_policy': 'Lossguide',
              'loss_function':trial.suggest_categorical("loss_function", ['MultiClass']),
              'bootstrap_type':trial.suggest_categorical("bootstrap_type", ['Poisson']),
              'use_best_model':True,
              'task_type':'GPU', 
#               'cat_features':"high",
              'learning_rate': trial.suggest_uniform('learning_rate' , 1e-5 , 1.0),
              'verbose':False,
             }
        clf = CatBoostClassifier(**params)
        clf.fit(x_train,y_train,eval_set=[(x_valid, y_valid)],early_stopping_rounds=100)
        preds = clf.predict_proba(x_valid)
        loss = log_loss(y_valid,preds)
        return loss

In [7]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2021-05-06 15:15:12,294] A new study created in memory with name: no-name-dfaf7e75-253d-4a8f-88e4-fbfb919b3f1a
[I 2021-05-06 15:16:33,970] Trial 0 finished with value: 1.0839115125738292 and parameters: {'depth': 10, 'l2_leaf_reg': 0.7784430887879386, 'loss_function': 'MultiClass', 'bootstrap_type': 'Poisson', 'learning_rate': 0.060662399774079355}. Best is trial 0 with value: 1.0839115125738292.
[I 2021-05-06 15:16:37,230] Trial 1 finished with value: 1.0889559508049789 and parameters: {'depth': 3, 'l2_leaf_reg': 1.0993785162004626, 'loss_function': 'MultiClass', 'bootstrap_type': 'Poisson', 'learning_rate': 0.7444438801207901}. Best is trial 0 with value: 1.0839115125738292.
[I 2021-05-06 15:16:41,670] Trial 2 finished with value: 1.0959667887264462 and parameters: {'depth': 5, 'l2_leaf_reg': 17.089888642867862, 'loss_function': 'MultiClass', 'bootstrap_type': 'Poisson', 'learning_rate': 0.9150963744292319}. Best is trial 0 with value: 1.0839115125738292.
[I 2021-05-06 15:16:47,15

In [8]:
study.best_value, study.best_params 

(1.0821429258192652,
 {'depth': 9,
  'l2_leaf_reg': 23.621286360583184,
  'loss_function': 'MultiClass',
  'bootstrap_type': 'Poisson',
  'learning_rate': 0.0027175782767777414})

In [9]:
optuna.visualization.plot_param_importances(study)

In [10]:
optuna.visualization.plot_optimization_history(study)

In [11]:
optuna.visualization.plot_parallel_coordinate(study)